In [4]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
from pathlib import Path

project_root = Path().resolve().parents[1]


In [7]:
from pathlib import Path

sys.path.append(str(Path().resolve().parents[1]))

In [43]:
import os
import re

import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from io import StringIO
import pandas as pd
import pickle
import igraph as ig

from grad.dataprep.util import create_pattern_dict, filter_dict
from grad.dataprep.util import *
from grad.dataprep.plotting_helpers import *
from grad.dataprep.generation import *
from util import *

# === CONFIG ===
current_dir = os.path.dirname(os.getcwd())
FILE_PATH = os.path.join(current_dir, "data", "IBM", "HI-SmallTransactions.txt")

current_dir = os.path.dirname(os.getcwd())
CSV_PATH = os.path.join(current_dir, "data", "IBM", "Hi-Small_Trans.csv")

# pattern_dict = create_pattern_dict(FILE_PATH)

# bipartite_data = filter_dict(pattern_dict, "BIPARTITE" )  # Example usage function

# df_data = pd.DataFrame(bipartite_data, columns=columns)
# df_data['src'] = df_data['From Bank'] + "_" + df_data['From Account']
# df_data['dst'] = df_data['To Bank'] + "_" + df_data['To Account']

df_full = pd.read_csv(CSV_PATH, dtype=str)
df_full["Is Laundering"] = pd.to_numeric(df_full["Is Laundering"], errors="coerce").fillna(2).astype(int)

df_full["From_Node"] = df_full["From Bank"].astype(str) + "_" + df_full["Account"].astype(str)
df_full["To_Node"] = df_full["To Bank"].astype(str) + "_" + df_full["Account.1"].astype(str)
df_full = df_full.rename(columns={"Account.1": "To Account"})

# full_data["Amount Paid"] = full_data["Amount Paid"].astype(float)
# full_data["Amount Received"] = full_data["Amount Received"].astype(float)
df = df_full[df_full['Is Laundering'] ==1]

START_ACCOUNT = "0222_812D127D0"


In [23]:
df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,From_Node,To_Node
4742,2022/09/01 00:21,070,100428660,001124,800825340,389769.39,US Dollar,389769.39,US Dollar,Cheque,1,070_100428660,001124_800825340
40621,2022/09/01 00:03,070,100428660,011474,805B716C0,29024.33,US Dollar,29024.33,US Dollar,Credit Card,1,070_100428660,011474_805B716C0
69152,2022/09/01 00:01,070,100428660,015980,80B39E7B0,792.92,US Dollar,792.92,US Dollar,Credit Card,1,070_100428660,015980_80B39E7B0
85763,2022/09/01 00:03,070,100428660,0113798,80DC756E0,13171425.53,US Dollar,13171425.53,US Dollar,Cheque,1,070_100428660,0113798_80DC756E0
90684,2022/09/01 00:23,070,100428660,0032375,80E480620,14288.83,US Dollar,14288.83,US Dollar,Cash,1,070_100428660,0032375_80E480620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5071168,2022/09/10 23:29,0111433,8065C2380,021550,8065C69E0,7773.42,US Dollar,7773.42,US Dollar,ACH,1,0111433_8065C2380,021550_8065C69E0
5073375,2022/09/10 23:07,009,80727F930,0219557,8072A28C0,448920.65,Ruble,448920.65,Ruble,Cheque,1,009_80727F930,0219557_8072A28C0
5074248,2022/09/12 03:34,0222,812D127D0,0050202,812D129C0,20085.71,Saudi Riyal,20085.71,Saudi Riyal,ACH,1,0222_812D127D0,0050202_812D129C0
5075056,2022/09/10 23:43,017327,8095F6C20,0015695,8095F6C70,3314.36,US Dollar,3314.36,US Dollar,ACH,1,017327_8095F6C20,0015695_8095F6C70


In [44]:
df_full

,Timestamp,From Bank,Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,From_Node,To_Node
0,2022/09/01 00:20,010,8000EBD30,010,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,010_8000EBD30,010_8000EBD30
1,2022/09/01 00:20,03208,8000F4580,001,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,03208_8000F4580,001_8000F5340
2,2022/09/01 00:00,03209,8000F4670,03209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,03209_8000F4670,03209_8000F4670
3,2022/09/01 00:02,012,8000F5030,012,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,012_8000F5030,012_8000F5030
4,2022/09/01 00:06,010,8000F5200,010,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,010_8000F5200,010_8000F5200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022/09/10 23:57,0054219,8148A6631,0256398,8148A8711,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0,0054219_8148A6631,0256398_8148A8711
5078341,2022/09/10 23:35,0015,8148A8671,0256398,8148A8711,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0,0015_8148A8671,0256398_8148A8711
5078342,2022/09/10 23:52,0154365,8148A6771,0256398,8148A8711,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0,0154365_8148A6771,0256398_8148A8711
5078343,2022/09/10 23:46,0256398,8148A6311,0256398,8148A8711,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0,0256398_8148A6311,0256398_8148A8711


In [60]:
keys = df_full["To Bank"].astype(str) + "|" + df_full["To Account"]
keys2 = df_full["From Bank"].astype(str) + "|" + df_full["Account"]

all_keys = pd.concat([keys, keys2])

codes, uniques = pd.factorize(all_keys)
mapping = dict(zip(uniques, range(len(uniques))))

df_full["To_Account_int"]   = keys.map(mapping)
df_full["From_Account_int"] = keys2.map(mapping)

In [61]:
df_full

,Timestamp,From Bank,Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,From_Node,To_Node,To_Account_int,From_Account_int
0,2022/09/01 00:20,010,8000EBD30,010,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,010_8000EBD30,010_8000EBD30,0,0
1,2022/09/01 00:20,03208,8000F4580,001,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,03208_8000F4580,001_8000F5340,1,420640
2,2022/09/01 00:00,03209,8000F4670,03209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,03209_8000F4670,03209_8000F4670,2,2
3,2022/09/01 00:02,012,8000F5030,012,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,012_8000F5030,012_8000F5030,3,3
4,2022/09/01 00:06,010,8000F5200,010,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,010_8000F5200,010_8000F5200,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022/09/10 23:57,0054219,8148A6631,0256398,8148A8711,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0,0054219_8148A6631,0256398_8148A8711,236079,417505
5078341,2022/09/10 23:35,0015,8148A8671,0256398,8148A8711,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0,0015_8148A8671,0256398_8148A8711,236079,333564
5078342,2022/09/10 23:52,0154365,8148A6771,0256398,8148A8711,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0,0154365_8148A6771,0256398_8148A8711,236079,236078
5078343,2022/09/10 23:46,0256398,8148A6311,0256398,8148A8711,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0,0256398_8148A6311,0256398_8148A8711,236079,265371


In [ ]:
def process_all_suspicious_networks_igraph(df):

    g = ig.Graph.DataFrame(
        df[["From_Account_int", "To_Account_int"]],
        directed=True,
        use_vids=True
    )

    components = g.components(mode="weak")

    networks = []
    
    suspicious_nodes = set(
        df.loc[df["Is Laundering"], "From_Account_int"]
    ).union(
        df.loc[df["Is Laundering"], "To_Account_int"]
    )

    for comp in components:
        vids = set(comp)

        if vids.isdisjoint(suspicious_nodes):
            continue     

        vids = list(comp)
        connected_nodes = set(vids)

        connected_df = df[
            df["From_Account_int"].isin(connected_nodes)
            & df["To_Account_int"].isin(connected_nodes)
        ]

        start_node = vids[0]

        networks.append({
            "start_node": start_node,
            "nodes": connected_nodes,
            "transactions": connected_df
        })

        print(
            f"Network from {start_node}: "
            f"{len(connected_nodes)} nodes, "
            f"{len(connected_df)} transactions"
        )

    return networks


In [76]:
df = df_full[df_full['Is Laundering'] ==1]

networks = process_all_suspicious_networks_igraph(df_full)

Network from 0: 372089 nodes, 4874569 transactions


In [77]:
import igraph as ig
from collections import deque

def extract_laundering_networks_igraph(
    df,
    max_depth=5,
    max_networks=10,
    collapse_threshold=10
):
    """
    Extract laundering-centered networks with surrounding non-laundering nodes
    using igraph for fast traversal.
    Returns a list of dictionaries (not graphs).
    """

    # --- Build full graph ---
    g = ig.Graph.DataFrame(
        df[["From_Account_int", "To_Account_int"]],
        directed=True,
        use_vids=True
    )

    # --- Identify laundering nodes ---
    laundering_nodes = set(
        df.loc[df["Is Laundering"] == 1, "From_Account_int"]
    ).union(
        df.loc[df["Is Laundering"] == 1, "To_Account_int"]
    )

    # --- Subgraph of laundering nodes only ---
    laundering_subgraph = g.subgraph(laundering_nodes)
    laundering_components = laundering_subgraph.components(mode="weak")

    networks = []
    seen_laundering_nodes = set()

    # --- Process each laundering component ---
    for comp in laundering_components:

        core_nodes = set(comp)

        # Skip if this laundering component was already processed
        if core_nodes & seen_laundering_nodes:
            continue

        # --- BFS expansion from all core laundering nodes ---
        visited = {}
        collapsed_nodes = set()

        queue = deque((v, 0) for v in core_nodes)

        while queue:
            node, depth = queue.popleft()

            if node in visited:
                continue

            visited[node] = depth

            neighbors = set(g.neighbors(node, mode="all"))

            # Collapse highly connected nodes
            if len(neighbors) > collapse_threshold:
                collapsed_nodes.add(node)
                continue

            # Depth control
            if depth >= max_depth:
                continue

            for nbr in neighbors:
                if nbr not in visited:
                    queue.append((nbr, depth + 1))

        component_nodes = set(visited.keys())

        # --- Extract transactions inside this network ---
        transactions = df[
            df["From_Account_int"].isin(component_nodes) &
            df["To_Account_int"].isin(component_nodes)
        ].copy()

        # --- Track laundering nodes in this component ---
        laundering_in_component = core_nodes

        start_node = next(iter(core_nodes))

        network = {
            "start_node": start_node,
            "nodes": component_nodes,
            "laundering_nodes": laundering_in_component,
            "collapsed_nodes": collapsed_nodes,
            "node_depths": visited,
            "transactions": transactions
        }

        networks.append(network)

        seen_laundering_nodes.update(core_nodes)

        print(
            f"Laundering network from {start_node}: "
            f"{len(component_nodes)} nodes, "
            f"{len(transactions)} transactions, "
            f"{len(collapsed_nodes)} collapsed hubs"
        )

        if len(networks) >= max_networks:
            break

    return networks


In [78]:
extract_laundering_networks_igraph(
    df_full,
    max_depth=4,
    max_networks=100,
    collapse_threshold=10
) 

Laundering network from 0: 2 nodes, 2 transactions, 2 collapsed hubs
Laundering network from 2: 229 nodes, 2502 transactions, 32 collapsed hubs
Laundering network from 3: 91455 nodes, 1448680 transactions, 5384 collapsed hubs
Laundering network from 10: 178 nodes, 1672 transactions, 23 collapsed hubs
Laundering network from 26: 3 nodes, 6 transactions, 2 collapsed hubs
Laundering network from 3993: 227 nodes, 2618 transactions, 19 collapsed hubs
Laundering network from 4720: 551 nodes, 6213 transactions, 76 collapsed hubs
Laundering network from 5487: 613 nodes, 6563 transactions, 67 collapsed hubs
Laundering network from 1984: 1014 nodes, 12045 transactions, 107 collapsed hubs
Laundering network from 42: 116 nodes, 990 transactions, 13 collapsed hubs
Laundering network from 5555: 6 nodes, 11 transactions, 2 collapsed hubs
Laundering network from 3523: 228 nodes, 2442 transactions, 26 collapsed hubs
Laundering network from 6017: 102 nodes, 946 transactions, 12 collapsed hubs
Laundering

[{'start_node': 0,
  'nodes': {0, 1},
  'laundering_nodes': {0, 1},
  'collapsed_nodes': {0, 1},
  'node_depths': {0: 0, 1: 0},
  'transactions':                 Timestamp From Bank    Account To Bank To Account  \
  0        2022/09/01 00:20       010  8000EBD30     010  8000EBD30   
  1047784  2022/09/01 22:06       001  8000F5340     001  8000F5340   
  
          Amount Received Receiving Currency Amount Paid Payment Currency  \
  0               3697.34          US Dollar     3697.34        US Dollar   
  1047784           64.81          US Dollar       64.81        US Dollar   
  
          Payment Format  Is Laundering      From_Node        To_Node  \
  0         Reinvestment              0  010_8000EBD30  010_8000EBD30   
  1047784   Reinvestment              0  001_8000F5340  001_8000F5340   
  
           To_Account_int  From_Account_int  
  0                     0                 0  
  1047784               1                 1  },
 {'start_node': 2,
  'nodes': {2,
   48,
   

In [70]:
def plot_graph_from_dict(graph_dict, save_path=None, show=True, dpi=300):
    """
    Parameters
    ----------
    graph_dict : dict
        Contains 'start_node', 'nodes', 'transactions' (DataFrame)

    save_path : str or Path or None
        If provided, saves the figure to this path

    show : bool
        Whether to display the plot

    dpi : int
        Resolution for saved image
    """

    df = graph_dict["transactions"]
    all_nodes = set(graph_dict["nodes"])

    G = nx.DiGraph()

    laundering_edges = set()
    laundering_nodes = graph_dict.get("laundering_nodes", set())
    connected_nodes = set()

    # ---- Build graph ----
    for _, row in df.iterrows():
        src = row["From_Node"]
        dst = row["To_Node"]
        if src != dst:
            G.add_edge(src, dst)

        if row["Is Laundering"] == 1:
            laundering_edges.add((src, dst))
            # laundering_nodes.update([src, dst])

        if src != dst:
            connected_nodes.update([src, dst])

    collapsed_nodes = graph_dict.get("collapsed_nodes", set())

    # ---- Node colors ----
    node_colors = []
    for node in G.nodes():
        if node in laundering_nodes:
            node_colors.append("red")
        elif node in collapsed_nodes:
            node_colors.append("gray")
        else:
            node_colors.append("lightblue")

    # ---- Edge colors ----
    edge_colors = [
        "red" if edge in laundering_edges else "black"
        for edge in G.edges()
    ]

    # ---- Layout ----
    pos = nx.spring_layout(G, seed=42, k=0.4)

    # ---- Plot ----
    plt.figure(figsize=(14, 14))
    nx.draw_networkx_nodes(
        G, pos,
        node_color=node_colors,
        node_size=100,
        alpha=0.9
    )
    nx.draw_networkx_edges(
        G, pos,
        edge_color=edge_colors,
        arrows=True,
        alpha=0.7
    )

    # ---- Highlight start node ----
    start = graph_dict.get("start_node")
    if start in G:
        nx.draw_networkx_nodes(
            G, pos,
            nodelist=[start],
            node_color="yellow",
            node_size=100,
            edgecolors=edge_colors,
            linewidths=3
        )

    # nx.draw_networkx_labels(G, pos, font_size=5)

    plt.title("Transaction Network (Red = Laundering, Gray = Collapsed)", fontsize=14)
    plt.axis("off")

    # ---- Save if requested ----
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(save_path, dpi=dpi, bbox_inches="tight")

    if show:
        plt.show()
    else:
        plt.close()

In [72]:
plot_graph_from_dict(networks[0])

KeyboardInterrupt: 